## mobiCARE Cardio-Server 연동 가이드
Version 01.01

In [1]:
import os
import requests
from collections import OrderedDict
import json
# from utils import *

import time
import datetime
import socket
import urllib
import re

In [2]:
URL_BASE = 'http://13.125.86.65:8080/mobiCARE/cardio'
USER_ID = 'northstar01'
USER_PW = 'northstar01'
PW_ENCRYPTION = 0
GMT_CODE = 'GMT+0900'
DEVICE_KIND = 5
COUNTRY_CODE = 'KR'
COUNTRY_NAME = 'Korea'
TIMEZONE = 'Asia/Seoul'

auth_token = ''
ip_wan = ''
ip_lan = ''
request_user_id = 0
organization_id = ''
list_measurement = []
list_ecg = []

In [3]:
ip_lan = socket.gethostbyname(socket.gethostname())
ip_check = urllib.request.urlopen('http://checkip.dyndns.org').read().decode('utf-8')
ip_wan = re.search(re.compile(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}'),ip_check).group() 
print ('WAN ip : ' + ip_wan) 
print ('LAN ip : ' + ip_lan)

WAN ip : 14.40.50.110
LAN ip : 127.0.1.1


In [4]:
# Make Login message 
URL_API_LOGIN = URL_BASE + '/Account/Login'
HEADERS = {'Content-Type': 'application/json; charset=utf-8', 'SX-Client-IP': ip_wan}

JSON_LOGIN = OrderedDict()
JSON_LOGIN["id"] = USER_ID
JSON_LOGIN["password"] = USER_PW
JSON_LOGIN["encryption"] = PW_ENCRYPTION
JSON_LOGIN["systemTime"] = int(time.time())
JSON_LOGIN["requestDateTime"] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
JSON_LOGIN["gmtCode"] = GMT_CODE
JSON_LOGIN["deviceKind"] = DEVICE_KIND
JSON_LOGIN["countryCode"] = COUNTRY_CODE
JSON_LOGIN["countryName"] = COUNTRY_NAME
JSON_LOGIN["timezone"] = TIMEZONE
TEXT_LOGIN = json.dumps(JSON_LOGIN)
print(TEXT_LOGIN)

{"id": "northstar01", "password": "northstar01", "encryption": 0, "systemTime": 1570778007, "requestDateTime": "2019-10-11 16:13:27", "gmtCode": "GMT+0900", "deviceKind": 5, "countryCode": "KR", "countryName": "Korea", "timezone": "Asia/Seoul"}


In [5]:
print('POST request %s' % URL_API_LOGIN)
res = requests.post(URL_API_LOGIN, data=TEXT_LOGIN, headers=HEADERS)
res_code = res.status_code
res_result = False
response_dic = {}
if res_code == 200:
    try:
        response_dic = json.loads(res.text)
        if 'error' in response_dic.keys():     
            if response_dic["error"] == 0:
                res_result = True
                auth_token = response_dic["accessToken"]
                organization_id = response_dic['userAccount']["organizationId"]
                request_user_id = response_dic['userAccount']["userId"]
    except (ValueError, TypeError) as e:
        print ("post exception : ", e)
        print (res.text)
elif res_code == 404:
    print ("send error - 404, Retry send cancel.")

print (res_result, res_code)
print ('organization_id', organization_id, 'token', auth_token)
print (res.text)

POST request http://13.125.86.65:8080/mobiCARE/cardio/Account/Login
True 200
organization_id 8 token 1570778007927D8VRNFU6
{"result":true,"extra":null,"systemTime":1570778007,"error":0,"message":null,"accessToken":"1570778007927D8VRNFU6","refreshToken":null,"userAccount":{"userId":21,"id":"northstar01","password":null,"organizationId":8,"userCode":"NORTHSTARTMB","department":"Software","userLevel":2,"firstName":"Mb","lastName":"Lee","gender":1,"birthday":"1991-07-03","phoneNumber":"+842437326303","email":"mb.lee@northstar.co.kr","address":null,"deviceKind":3,"encryption":1,"status":0,"deactivate":0,"gmtCode":"GMT+0900","joinDateTime":"2019-09-10 11:13:40","updateDateTime":null,"deactivateDateTime":null},"organization":{"organizationId":8,"organizationCode":"H4D0B5QW","organizationName":"NorthStar","countryCode":"KR","countryName":"Korea","state":"Seoul","city":"Seongdong-gu","address":"51, Seongsui-ro, Seongdong-gu","phoneNumber":"+821035666371","contractLevel":3,"deactivate":0,"admini

In [6]:
# Make SelectPatientMeasurementInfoByReviewer message 
URL_API_LIST = URL_BASE + '/Measurement/SelectPatientMeasurementInfoByReviewer'
HEADERS = {'Content-Type': 'application/json; charset=utf-8',
           'SX-Auth-Token': auth_token,
           'SX-Client-IP': ip_wan }

JSON_LIST = OrderedDict()
JSON_LIST["requestUserId"] = request_user_id
JSON_LIST["organizationId"] = organization_id
JSON_LIST["startDateTime"] = '2019-01-01 00:00:00'
JSON_LIST["endDateTime"] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

JSON_LIST["systemTime"] = int(time.time())
JSON_LIST["requestDateTime"] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
JSON_LIST["gmtCode"] = GMT_CODE
JSON_LIST["deviceKind"] = DEVICE_KIND
JSON_LIST["countryCode"] = COUNTRY_CODE
JSON_LIST["countryName"] = COUNTRY_NAME
JSON_LIST["timezone"] = TIMEZONE
TEXT_LIST = json.dumps(JSON_LIST)
print(TEXT_LIST)

{"requestUserId": 21, "organizationId": 8, "startDateTime": "2019-01-01 00:00:00", "endDateTime": "2019-10-11 16:13:27", "systemTime": 1570778007, "requestDateTime": "2019-10-11 16:13:27", "gmtCode": "GMT+0900", "deviceKind": 5, "countryCode": "KR", "countryName": "Korea", "timezone": "Asia/Seoul"}


In [7]:
print('POST request %s' % URL_API_LIST)
res = requests.post(URL_API_LIST, data=TEXT_LIST, headers=HEADERS)
res_code = res.status_code
res_result = False
response_dic = {}
list_measurement = []
if res_code == 200:
    try:
        response_dic = json.loads(res.text)
        if 'error' in response_dic.keys():     
            if response_dic["error"] == 0:
                res_result = True
                list_measurement = response_dic["measurementInfoList"]
    except (ValueError, TypeError) as e:
        print ("post exception : ", e)
        print (res.text)
elif res_code == 404:
    print ("send error - 404, Retry send cancel.")

print (res_result, res_code)
print ('count measurement', len(list_measurement))
print (res.text)

POST request http://13.125.86.65:8080/mobiCARE/cardio/Measurement/SelectPatientMeasurementInfoByReviewer
True 200
count measurement 1
{"result":true,"extra":null,"systemTime":1570778007,"error":0,"message":null,"measurementInfoList":[{"measurementId":110,"measurementCode":"NSTest","registerUserId":21,"reviewerUserId":21,"deviceCode":"29410","organizationId":8,"patientCode":"SEERSWESLEY","diseaseCode":null,"firstName":"Wesley","lastName":"Kwon","gender":1,"birthday":"1983-03-13","height":180.0,"weight":68.0,"phoneNumber":null,"measurementStatus":6,"shard":"2019","part":29,"reanalyzing":0,"deviceKind":3,"elapsedTime":15593,"gmtCode":"GMT+0900","timezone":null,"countryCode":null,"dateTime":"2019-08-30 14:19:36","startDateTime":"2019-08-30 14:25:05","endDateTime":"2019-08-30 18:44:58","modifyDateTime":"2019-09-20 10:21:07","etc":"Wesley Riding Check"}]}


In [8]:
# Make SelectECGData  message 
URL_API_ECG = URL_BASE + '/Measurement/SelectECGData'
HEADERS = {'Content-Type': 'application/json; charset=utf-8',
           'SX-Auth-Token': auth_token,
           'SX-Client-IP': ip_wan }

JSON_ECG = OrderedDict()
JSON_ECG["requestUserId"] = request_user_id
JSON_ECG["organizationId"] = organization_id
JSON_ECG["measurementCode"] = ''

JSON_ECG["systemTime"] = int(time.time())
JSON_ECG["requestDateTime"] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
JSON_ECG["gmtCode"] = GMT_CODE
JSON_ECG["deviceKind"] = DEVICE_KIND
JSON_ECG["countryCode"] = COUNTRY_CODE
JSON_ECG["countryName"] = COUNTRY_NAME
JSON_ECG["timezone"] = TIMEZONE
TEXT_ECG = json.dumps(JSON_ECG)
print(TEXT_ECG)

{"requestUserId": 21, "organizationId": 8, "measurementCode": "", "systemTime": 1570778007, "requestDateTime": "2019-10-11 16:13:27", "gmtCode": "GMT+0900", "deviceKind": 5, "countryCode": "KR", "countryName": "Korea", "timezone": "Asia/Seoul"}


In [9]:
for idx, measurement in enumerate(list_measurement):
    measurement_code = measurement['measurementCode']
    JSON_ECG["measurementCode"] = measurement_code
    TEXT_ECG = json.dumps(JSON_ECG)
    
    print('POST request %s' % URL_API_ECG)
    res = requests.post(URL_API_ECG, data=TEXT_ECG, headers=HEADERS)
    res_code = res.status_code
    res_result = False
    response_dic = {}
    if res_code == 200:
        try:
            response_dic = json.loads(res.text)
            if 'error' in response_dic.keys():     
                if response_dic["error"] == 0:
                    res_result = True
                    list_measurement[idx]["ecgDataList"] = response_dic["ecgDataList"]
        except (ValueError, TypeError) as e:
            print ("post exception : ", e)
            print (res.text)
    elif res_code == 404:
        print ("send error - 404, Retry send cancel.")

    print (res_result, res_code)
    print ('count ecg list', len(list_measurement[idx]["ecgDataList"]))
    print (res.text)
    print ('')


POST request http://13.125.86.65:8080/mobiCARE/cardio/Measurement/SelectECGData
True 200
count ecg list 1
{"result":true,"extra":null,"systemTime":1570778007,"error":0,"message":null,"ecgDataList":[{"ecgId":203,"organizationId":8,"measurementCode":"NSTest","key":null,"md5sum":"d9e7c1468fdf71aa162b585f28ed84","sequenceNumber":1,"sampling":256,"gain":100.0,"endian":1,"deviceKind":2,"gmtCode":"GMT+0900","startDateTime":"2019-08-30 14:25:05","endDateTime":"2019-08-30 18:44:58","uploadDateTime":"2019-09-02 09:03:48","modifyDateTime":null,"fileSize":7984384}]}



In [10]:
# Make DownloadECGFile message 
URL_API_DOWNLOAD_ECG = URL_BASE + '/Measurement/DownloadECGFile/{code}:{id}'
HEADERS = {'Content-Type': 'application/json; charset=utf-8',
           'SX-Auth-Token': auth_token,
           'SX-Client-IP': ip_wan }

In [11]:
for idx, measurement in enumerate(list_measurement):
    measurement_code = measurement['measurementCode']
    for idx, ecg in enumerate(measurement['ecgDataList']):
        ecg_id = ecg['ecgId']
        URL_API_DOWNLOAD_ECG = URL_BASE \
            + '/Measurement/DownloadECGFile/{code}:{id}'.format(code=measurement_code, 
                                                                id=str(ecg_id))
        print (measurement_code, ecg_id)
        print (URL_API_DOWNLOAD_ECG)
        
        res = requests.get(URL_API_DOWNLOAD_ECG, stream=True, headers=HEADERS)
        res_code = res.status_code
        res_result = False
        if res_code == 200:
            # print (res.headers)
            content_disposition = res.headers['Content-Disposition']
            print (content_disposition)
            filename = re.findall('filename=.*"([^ ]*)"',content_disposition)[0]
            print('filename : ', filename)

            with open(filename, 'wb') as f:
                for chunk in res.iter_content(chunk_size=512):
                    if chunk:  # filter out keep-alive new chunks
                        f.write(chunk)
        elif res_code == 404:
            print ("send error - 404, Retry send cancel.")
        

NSTest 203
http://13.125.86.65:8080/mobiCARE/cardio/Measurement/DownloadECGFile/NSTest:203
form-data; name="attachment"; filename="NSTest_01_20190830_142505.ecg"
filename :  NSTest_01_20190830_142505.ecg


In [12]:
# Make SelectAnalysisOutputAnnotationFile  message 

URL_API_ANNOTATION = URL_BASE + '/Analysis/SelectAnalysisOutputAnnotationFile'
HEADERS = {'Content-Type': 'application/json; charset=utf-8',
           'SX-Auth-Token': auth_token,
           'SX-Client-IP': ip_wan }

JSON_ANNOTATION = OrderedDict()
JSON_ANNOTATION["requestUserId"] = request_user_id
JSON_ANNOTATION["organizationId"] = organization_id
JSON_ANNOTATION["measurementCode"] = ''

JSON_ANNOTATION["systemTime"] = int(time.time())
JSON_ANNOTATION["requestDateTime"] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
JSON_ANNOTATION["gmtCode"] = GMT_CODE
JSON_ANNOTATION["deviceKind"] = DEVICE_KIND
JSON_ANNOTATION["countryCode"] = COUNTRY_CODE
JSON_ANNOTATION["countryName"] = COUNTRY_NAME
JSON_ANNOTATION["timezone"] = TIMEZONE
TEXT_ANNOTATION = json.dumps(JSON_ANNOTATION)
print(TEXT_ANNOTATION)

{"requestUserId": 21, "organizationId": 8, "measurementCode": "", "systemTime": 1570778008, "requestDateTime": "2019-10-11 16:13:28", "gmtCode": "GMT+0900", "deviceKind": 5, "countryCode": "KR", "countryName": "Korea", "timezone": "Asia/Seoul"}


In [13]:
for idx, measurement in enumerate(list_measurement):
    measurement_code = measurement['measurementCode']
    JSON_ANNOTATION["measurementCode"] = measurement_code
    TEXT_ANNOTATION = json.dumps(JSON_ANNOTATION)

    print('POST request %s' % URL_API_ECG)
    res = requests.post(URL_API_ANNOTATION, data=TEXT_ANNOTATION, headers=HEADERS)
    res_code = res.status_code
    res_result = False
    response_dic = {}
    if res_code == 200:
        try:
            response_dic = json.loads(res.text)
            if 'error' in response_dic.keys():     
                if response_dic["error"] == 0:
                    res_result = True
                    list_measurement[idx]["analysisOutputFileList"] = response_dic["analysisOutputFileList"]
        except (ValueError, TypeError) as e:
            print ("post exception : ", e)
            print (res.text)
    elif res_code == 404:
        print ("send error - 404, Retry send cancel.")

    print (res_result, res_code)
    print ('count annotation list', len(list_measurement[idx]["analysisOutputFileList"]))
    print (res.text)
    print ('')

POST request http://13.125.86.65:8080/mobiCARE/cardio/Measurement/SelectECGData
True 200
count annotation list 1
{"result":true,"extra":null,"systemTime":1570778008,"error":0,"message":null,"analysisOutputFileList":[{"analysisOutputFileId":5474,"organizationId":8,"measurementCode":"NSTest","fileKey":null,"md5sum":"2b59dc42c6c9a1072c9817f372398a0","fileType":7,"reanalyzing":0,"uploadDateTime":"2019-09-20 10:20:19"}]}



In [14]:
# Make DownloadAnaysisOutputFile message 
URL_API_DOWNLOAD_ANNOTATION = URL_BASE + '/Analysis/DownloadAnalysisOutputFile/{code}:{id}'
HEADERS = {'Content-Type': 'application/json; charset=utf-8',
           'SX-Auth-Token': auth_token,
           'SX-Client-IP': ip_wan }

In [15]:
for idx, measurement in enumerate(list_measurement):
    measurement_code = measurement['measurementCode']
    for idx, analysis in enumerate(measurement['analysisOutputFileList']):
        analysis_id = analysis['analysisOutputFileId']
        URL_API_DOWNLOAD_ANNOTATION = URL_BASE \
            + '/Analysis/DownloadAnalysisOutputFile/{code}:{id}'.format(code=measurement_code,
                                                                        id=str(analysis_id))
        print (measurement_code, analysis_id)
        print (URL_API_DOWNLOAD_ANNOTATION)
        
        res = requests.get(URL_API_DOWNLOAD_ANNOTATION, stream=True, headers=HEADERS)
        res_code = res.status_code
        res_result = False
        if res_code == 200:
            # print (res.headers)
            content_disposition = res.headers['Content-Disposition']
            print (content_disposition)
            filename = re.findall('filename=.*"([^ ]*)"',content_disposition)[0]
            print('filename : ', filename)

            with open(filename, 'wb') as f:
                for chunk in res.iter_content(chunk_size=512):
                    if chunk:  # filter out keep-alive new chunks
                        f.write(chunk)
        elif res_code == 404:
            print ("send error - 404, Retry send cancel.")

NSTest 5474
http://13.125.86.65:8080/mobiCARE/cardio/Analysis/DownloadAnalysisOutputFile/NSTest:5474
form-data; name="attachment"; filename="hbclass.txt"
filename :  hbclass.txt


In [16]:
!pwd
!ls -al

list_measurement

/home/nsc/seers/seers_cai_mb/notebook
합계 22544
drwxrwxr-x 3 nsc nsc     4096 10월 11 16:12 .
drwxrwxr-x 7 nsc nsc     4096 10월 11 13:36 ..
drwxrwxr-x 2 nsc nsc     4096 10월 11 10:31 .ipynb_checkpoints
-rw-rw-r-- 1 nsc nsc  7984384 10월 11 16:13 NSTest_01_20190830_142505.ecg
-rw-rw-r-- 1 nsc nsc 10728643 10월 10 23:10 dataset_viewer.ipynb
-rw-rw-r-- 1 nsc nsc   345696 10월 11 16:13 hbclass.txt
-rw-rw-r-- 1 nsc nsc    18197 10월 11 16:12 load_mc100.ipynb
-rw-rw-r-- 1 nsc nsc    46883 10월 11 16:12 mc100_api.ipynb
-rw-rw-r-- 1 nsc nsc    27114 10월 11 13:46 mc100_class.ipynb
-rw-rw-r-- 1 nsc nsc   204766 10월 10 23:01 preprocess-db1.ipynb
-rw-rw-r-- 1 nsc nsc  2190680 10월 10 22:50 preprocess-db2.ipynb
-rw-rw-r-- 1 nsc nsc  1510966 10월  7 13:36 seers_demo.ipynb


[{'measurementId': 110,
  'measurementCode': 'NSTest',
  'registerUserId': 21,
  'reviewerUserId': 21,
  'deviceCode': '29410',
  'organizationId': 8,
  'patientCode': 'SEERSWESLEY',
  'diseaseCode': None,
  'firstName': 'Wesley',
  'lastName': 'Kwon',
  'gender': 1,
  'birthday': '1983-03-13',
  'height': 180.0,
  'weight': 68.0,
  'phoneNumber': None,
  'measurementStatus': 6,
  'shard': '2019',
  'part': 29,
  'reanalyzing': 0,
  'deviceKind': 3,
  'elapsedTime': 15593,
  'gmtCode': 'GMT+0900',
  'timezone': None,
  'countryCode': None,
  'dateTime': '2019-08-30 14:19:36',
  'startDateTime': '2019-08-30 14:25:05',
  'endDateTime': '2019-08-30 18:44:58',
  'modifyDateTime': '2019-09-20 10:21:07',
  'etc': 'Wesley Riding Check',
  'ecgDataList': [{'ecgId': 203,
    'organizationId': 8,
    'measurementCode': 'NSTest',
    'key': None,
    'md5sum': 'd9e7c1468fdf71aa162b585f28ed84',
    'sequenceNumber': 1,
    'sampling': 256,
    'gain': 100.0,
    'endian': 1,
    'deviceKind': 2,
 

In [17]:
# Make SelectAnalysisBeatClassify message 
URL_API_ANALYSIS = URL_BASE + '/Analysis/SelectAnalysisBeatClassify'
HEADERS = {'Content-Type': 'application/json; charset=utf-8',
           'SX-Auth-Token': auth_token,
           'SX-Client-IP': ip_wan }

JSON_ANALYSIS = OrderedDict()
JSON_ANALYSIS["requestUserId"] = request_user_id
JSON_ANALYSIS["organizationId"] = organization_id
JSON_ANALYSIS["measurementCode"] = measurement_code
JSON_ANALYSIS["includeNoise"] = True

JSON_ANALYSIS["systemTime"] = int(time.time())
JSON_ANALYSIS["requestDateTime"] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
JSON_ANALYSIS["gmtCode"] = GMT_CODE
JSON_ANALYSIS["deviceKind"] = DEVICE_KIND
JSON_ANALYSIS["countryCode"] = COUNTRY_CODE
JSON_ANALYSIS["countryName"] = COUNTRY_NAME
JSON_ANALYSIS["timezone"] = TIMEZONE
TEXT_ANALYSIS = json.dumps(JSON_ANALYSIS)
print(TEXT_ANALYSIS)

{"requestUserId": 21, "organizationId": 8, "measurementCode": "NSTest", "includeNoise": true, "systemTime": 1570778008, "requestDateTime": "2019-10-11 16:13:28", "gmtCode": "GMT+0900", "deviceKind": 5, "countryCode": "KR", "countryName": "Korea", "timezone": "Asia/Seoul"}


In [18]:
print('POST request %s' % URL_API_ANALYSIS)
res = requests.post(URL_API_ANALYSIS, data=TEXT_ANALYSIS, headers=HEADERS)
res_code = res.status_code
res_result = False
response_dic = {}
if res_code == 200:
    try:
        response_dic = json.loads(res.text)
        if 'error' in response_dic.keys():     
            if response_dic["error"] == 0:
                res_result = True
                analysis_list = response_dic["analysisBeatClassifySimpleList"]

    except (ValueError, TypeError) as e:
        print ("post exception : ", e)
        print (res.text)
elif res_code == 404:
    print ("send error - 404, Retry send cancel.")

print (res_result, res_code)
# print (analysis_list)
for beat in analysis_list:
    if beat['beatClass'] != 'N':
        print(beat)

POST request http://13.125.86.65:8080/mobiCARE/cardio/Analysis/SelectAnalysisBeatClassify
True 200
{'analysisBeatClassifyId': 179471, 'seconds': 0.035, 'beatClass': 'V', 'beatShape': 1, 'modifyFlag': 0, 'removeFlag': 0, 'noise': 0}
{'analysisBeatClassifyId': 179472, 'seconds': 13.953, 'beatClass': 'V', 'beatShape': 2, 'modifyFlag': 0, 'removeFlag': 0, 'noise': 0}
{'analysisBeatClassifyId': 179473, 'seconds': 14.504, 'beatClass': 'V', 'beatShape': 3, 'modifyFlag': 0, 'removeFlag': 0, 'noise': 0}
{'analysisBeatClassifyId': 179475, 'seconds': 20.156, 'beatClass': 'V', 'beatShape': 5, 'modifyFlag': 0, 'removeFlag': 0, 'noise': 0}
{'analysisBeatClassifyId': 182530, 'seconds': 2479.777, 'beatClass': 'S', 'beatShape': 21, 'modifyFlag': 0, 'removeFlag': 0, 'noise': 0}
{'analysisBeatClassifyId': 183022, 'seconds': 2807.063, 'beatClass': 'S', 'beatShape': 26, 'modifyFlag': 0, 'removeFlag': 0, 'noise': 0}
{'analysisBeatClassifyId': 183103, 'seconds': 2861.188, 'beatClass': 'S', 'beatShape': 27, '